In [ ]:
import torch 
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2 as cv

import os
from pathlib import Path

import timm
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

# from tqdm import tqdm
from tqdm.notebook import trange, tqdm

In [ ]:
DEVICE="mps" if torch.backends.mps.is_built() else "cpu"
DEVICE

In [ ]:
class args:
    epoch = 20
    seed = 2024

In [ ]:
df_dir = "/Users/yhemmy/Documents/code/hotel-id-experiments/dataset/randomHotels/randomHotelsFeats.csv"
df_pikcle_dir = "/Users/yhemmy/Documents/code/hotel-id-experiments/dataset/randomHotels/randomHotelsFeats.pkl"

In [ ]:
# df = pd.read_csv(df_dir,converters={"hsv_feats":pd.eval,"rgb_feats":pd.eval,"hist_feats":pd.eval})
df = pd.read_pickle(df_pikcle_dir)
df.head()

In [ ]:
df.dtypes

In [ ]:
df = df.astype({"hotel_id":"str"})
df.dtypes

In [ ]:
df.head()

In [ ]:
df.shape[0]==df.image_id.nunique()

# Dataset Class
### Moved to a script

# Data split & Dataloader

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.head()
df.shape
hotel_image_count = df.groupby("hotel_id")["image_id"].count()
validation_hotels =  hotel_image_count[hotel_image_count>1]
validation_hotels
# validation_hotels.index
# df["hotel_id"].isin(validation_hotels.index)
validation_data = df[df["hotel_id"].isin(validation_hotels.index)]
validation_df = validation_data.groupby("hotel_id").sample(1,random_state=2024)
# validation_df.shape
train_df = df[~df["image_id"].isin(validation_df["image_id"])]
# train_df.shape[0]+validation_df.shape[0]
print(f"Train data sample: {train_df.shape[0]} \nValidation data sample: {validation_df.shape[0]}")


In [ ]:
train_df.head()

In [ ]:
validation_df.head()

In [ ]:
validation_df = validation_df.reset_index(drop=True)

In [ ]:
train_imgs_dir = []
for ind,row in train_df.iterrows():
    image_id = row["image_id"]
    hotel_id = row["hotel_id"]
    path = row["path"]

    train_imgs_dir.append((path,hotel_id,image_id))

validation_imgs_dir = []
for ind,row in validation_df.iterrows():
    image_id = row["image_id"]
    hotel_id = row["hotel_id"]
    path = row["path"]

    validation_imgs_dir.append((path,hotel_id,image_id))

In [ ]:
sample = cv.imread(train_imgs_dir[500][0])
sample = cv.resize(sample,(224,224))
print(sample.shape)
plt.imshow(sample)

In [ ]:
sample = cv.imread(validation_imgs_dir[500][0])
sample = cv.resize(sample,(224,224))
print(sample.shape)
plt.imshow(sample)

In [ ]:
unique_labels = df["hotel_id"].unique()
len(unique_labels)

In [ ]:
from utils.hotelsDataLoader import HOTELS
    
train_dataset = HOTELS(train_imgs_dir,unique_labels)
validation_dataset = HOTELS(validation_imgs_dir,unique_labels)

train_dataloader = DataLoader(
    train_dataset,
    num_workers =1,
    batch_size = 32,
    shuffle = True
)
validation_dataloader = DataLoader(
    validation_dataset,
    num_workers =1,
    batch_size = 32,
    shuffle = False
)

# x, y,img_id = next(iter(train_dataloader))




In [ ]:
# validation_dataset.paths
# validation_imgs_dir

In [ ]:
for i,(x, y,img_id) in enumerate(validation_dataloader):
    print(x)
    print(y)
    print(img_id)
    break

In [ ]:
# # test onehot encoding
# t = train_imgs_dir[12000][1]
# list(train_dataset.get_one_hot_encoding(t)).index(True)

In [ ]:
print(f"A batch holds {len(x)},{len(y)},{len(img_id)} of images, label and id respectively",)

In [ ]:
x.shape,x[0].shape

In [ ]:
y.shape

# Model

In [ ]:
# Features size
rgb_size = len(df["rgb_feats"][0])
hsv_size = len(df["hsv_feats"][0])
hist_size = len(df["hist_feats"][0])

num_classes=len(unique_labels)

In [ ]:
class EmbeddingModel(nn.Module):
    def __init__(self, num_classes,features_dataframe,rgb_size,hsv_size,hist_size,embedding_size=128,
                 backbone_name="efficientnet_b0"):
        super().__init__()
        
        self.features_dataframe = features_dataframe 
        # self.rgb_size = rgb_size
        # self.hsv_size = hsv_size
        # self.hist_size = hist_size   

        self.num_classes = num_classes 
        self.backbone = timm.create_model(model_name = backbone_name,num_classes=num_classes, pretrained = True)
        in_features = self.backbone.get_classifier().in_features

        self.backbone.classifier = nn.Identity()
        self.embedding = nn.Linear(in_features, embedding_size)
        self.classifier = nn.Linear(embedding_size,num_classes)
        self.rgbClassifier = nn.Linear(rgb_size+embedding_size,num_classes)
        self.hsvClassifier = nn.Linear(hsv_size+embedding_size,num_classes)
        self.histClassifier = nn.Linear(hist_size+embedding_size,num_classes)

    def forward(self,x):
            """
            Return embeddings
            """
            x = self.backbone(x)
            x = x.view(x.size(0),-1)
            x = self.embedding(x)
            return x
    
    def extractColorFeatures(self,image_ids,feat="rgb_feats"):
        """
        return color features
        """
        color_feature = []
        for img_id in image_ids:
            color_feats = df[df.image_id==img_id][feat].values[0]
            color_feature.append(color_feats)
        return color_feature

    def fuseFeatures(self,features_embedding,features_color):
        """
        return fused features i.e. embedding + color_features
        """
        fused_features = []
        for i,colorFeats in enumerate(features_color):
            colorFeats =torch.tensor(colorFeats,dtype=torch.float).to(DEVICE)
            embedding = features_embedding[i]
            features = torch.cat((embedding,colorFeats))
            fused_features.append(features)
        # fused_features = torch.stack(fused_features,0)
        return torch.stack(fused_features)

    def classifyWithEmbedding(self,x):
        """
        return hotel class using just embeddings
        """
        hotel_class = self.classifier(x)
        return hotel_class

    def classifyWithFusedFeatures(self,fused_features,classifer_to_use):
        """
        return hotel class using improved embeddings 
        """
        if classifer_to_use=="rgb":
            hotel_class = self.rgbClassifier(fused_features)
            return hotel_class
        elif classifer_to_use=="hsv":
            hotel_class = self.hsvClassifier(fused_features)
            return hotel_class
        else:
            hotel_class = self.histClassifier(fused_features)
            return hotel_class
         

    
model = EmbeddingModel(num_classes,df,rgb_size,hsv_size,hist_size).to(DEVICE)
# emb_ffff = model(torch.zeros((1, 3, 224, 224)).to(DEVICE))
# print(emb_ffff.shape)
# test_fussion = emb_ffff


# Model helper Funcs

In [ ]:
def generateFeatures(dataloader,model,improveEmbedding = False,colorFeat= None):
    features_all= []
    target_all=[]

    model.eval()
    with torch.no_grad():
        bar_description = "Generating embedding..."
        if(improveEmbedding):
             bar_description = "Extracting & improving embedding with {colorFeat}..."

        dataloader = tqdm(dataloader,desc=bar_description)
        for batch_no,(x, y,img_ids) in enumerate(dataloader):
                x = x.to(DEVICE)
                y = y.to(DEVICE)
                x = model(x)
                if(colorFeat):
                    color_feats = model.extractColorFeatures(img_ids,colorFeat)
                    x = model.fuseFeatures(x,color_feats)
                    target_all.extend(y.cpu().numpy())
                    features_all.extend(x.detach().cpu().numpy())
                else:
                    target_all.extend(y.cpu().numpy())
                    features_all.extend(x.detach().cpu().numpy())
                break
    target_all = np.array(target_all).astype(np.float32)
    features_all = np.array(features_all).astype(np.float32)
    return features_all,target_all

In [ ]:
OUTPUT_FOLDER ="/Users/yhemmy/Documents/code/hotel-id-experiments/models/"
def save_checkpoint(model, scheduler, optimizer, epoch, name, loss=None, score=None):
    checkpoint = {"epoch": epoch,
                  "model": model.state_dict(),
                  "scheduler": scheduler.state_dict(),
                  "optimizer": optimizer.state_dict(),
                  "loss": loss,
                  "score": score,
                  }

    torch.save(checkpoint, f"{OUTPUT_FOLDER}checkpoint-{name}.pt")


def load_checkpoint(model, scheduler, optimizer, name):
    checkpoint = torch.load(f"{OUTPUT_FOLDER}checkpoint-{name}.pt")

    model.load_state_dict(checkpoint["model"])
    scheduler.load_state_dict(checkpoint["scheduler"])
    return model, scheduler, optimizer, checkpoint["epoch"]

In [ ]:
def decode_one_hot(y_one_hot):
    y = np.argmax(y_one_hot.cpu().numpy(), axis=1)
    return y

In [ ]:
decode_one_hot(y)

In [ ]:
N_MATCHES = 5
def test_classification(loader, model,colorFeat= None):
    targets_all = []
    outputs_all = []
    outputs = None
    
    model.eval()
    dataloader = tqdm(loader, desc="Classification")
    
    for batch_no,(x, y,img_ids) in enumerate(dataloader):
        x = x.to(DEVICE)
        y = decode_one_hot(y)
        x = model(x)
        #improve embedding
        if(colorFeat and colorFeat=="rgb_feats"):
            color_feats = model.extractColorFeatures(img_ids,colorFeat)
            x = model.fuseFeatures(x,color_feats)
            outputs = model.rgbClassifier(x)
        elif(colorFeat and colorFeat=="hsv_feats"):
            color_feats = model.extractColorFeatures(img_ids,colorFeat)
            x = model.fuseFeatures(x,color_feats)
            outputs = model.hsvClassifier(x)

        elif(colorFeat and colorFeat=="hist_feats"):
            color_feats = model.extractColorFeatures(img_ids,colorFeat)
            x = model.fuseFeatures(x,color_feats)
            outputs = model.histClassifier(x)
        #use only embedding
        else:
            outputs = model.classifier(x)
        targets_all.extend(y)
        outputs_all.extend(torch.sigmoid(outputs).detach().cpu().numpy())
            
        
    
    # repeat targets to N_MATCHES for easy calculation of MAP@5
    y = np.repeat([targets_all], repeats=N_MATCHES, axis=0).T
    # sort predictions in ascending order i.e least class to top class
    sorted_indices = np.array(np.argsort(np.array(outputs_all),axis=1))
    # flip to sort in descending order and get top 5 classes i.e top class to least class 
    preds = np.flip(sorted_indices,1)[:,:N_MATCHES]
    preds = np.argsort(-np.array(outputs_all), axis=1)[:, :N_MATCHES]
    # check if any of top 5 predictions are correct and calculate mean accuracy
    acc_top_5 = (preds == y).any(axis=1).mean()
    # calculate prediction accuracy
    acc_top_1 = np.mean(targets_all == np.argmax(outputs_all, axis=1))

    print(f"Classification accuracy: {acc_top_1:0.4f}, MAP@5: {acc_top_5:0.4f}")
    return acc_top_1, acc_top_5

In [ ]:
target = np.repeat([[2,1]],2,0).T
target
# np.argsort(-np.array([1,2,3,4]))

In [ ]:
-np.array([1,2,3,4])

In [ ]:
# torch.sigmoid(torch.tensor([[0.5,0.2,0.3],[0.4,0.4,0.2]]))
sorted_indices = np.argsort(np.array(torch.sigmoid(torch.tensor([[0.5,0.2,0.3],[0.4,0.4,0.2]]))),-1)
sorted_indices
pred = np.flip(sorted_indices,1)[:,:2]
pred

In [ ]:
(pred == target).any(axis=1).mean()

In [ ]:
# f_all,t_all =generateFeatures(train_dataloader,model,improveEmbedding=True)

In [ ]:
# f_all.shape

# Train Function with Color+Embedding Features

In [ ]:
def trainEpoch(dataloader,model,criterion, optimizer, scheduler, epoch,classifier_to_use):
    targets_all=[]
    predicts_all = []
    losses = []

    model.train()
    t = tqdm(dataloader)

    for batch_no,(x, y,img_ids) in enumerate(t):
        optimizer.zero_grad()
        x = x.to(DEVICE)
        y = y.to(DEVICE)
    
        x = model(x)
        if(classifier_to_use=="rgb"):
            color_feats = model.extractColorFeatures(img_ids,feat="rgb_feats")
            x = model.fuseFeatures(x,color_feats)
            outputs = model.rgbClassifier(x)
            loss = criterion(outputs,y)

            loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()

            losses.append(loss.item())
            targets_all.extend(np.argmax(y.cpu().numpy(), axis=1))
            predicts_all.extend(torch.sigmoid(outputs).detach().cpu().numpy())

        elif(classifier_to_use=="hsv"):
            color_feats = model.extractColorFeatures(img_ids,feat="hsv_feats")
            x = model.fuseFeatures(x,color_feats)
            outputs = model.hsvClassifier(x)
            loss = criterion(outputs,y)

            loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()

            losses.append(loss.item())
            targets_all.extend(np.argmax(y.cpu().numpy(), axis=1))
            predicts_all.extend(torch.sigmoid(outputs).detach().cpu().numpy())

        elif(classifier_to_use=="hist"):
            color_feats = model.extractColorFeatures(img_ids,feat="hist_feats")
            x = model.fuseFeatures(x,color_feats)
            outputs = model.histClassifier(x)
            loss = criterion(outputs,y)

            loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()

            losses.append(loss.item())
            targets_all.extend(np.argmax(y.cpu().numpy(), axis=1))
            predicts_all.extend(torch.sigmoid(outputs).detach().cpu().numpy())

        else:
            # classifier_to_use=="embedding"
            outputs = model.classifyWithEmbedding(x)
            loss = criterion(outputs,y)

            loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()

            losses.append(loss.item())
            targets_all.extend(np.argmax(y.cpu().numpy(), axis=1))
            predicts_all.extend(torch.sigmoid(outputs).detach().cpu().numpy())
        


        score = np.mean(targets_all == np.argmax(predicts_all, axis=1))
        desc = f"Training epoch {epoch}/{20} - batch loss:{loss:0.4f}, accuracy: {score:0.4f}"
        t.set_description(desc)
        
    return np.mean(losses), score

In [ ]:
# np.mean(np.array([[2,3,4,3]])==np.array([[2,3,4,5]]),axis=1)
# torch.sigmoid(torch.tensor([2,3,4]))
# np.array([[1,2]
#           ,[5,3]]).argmax(1)
# df.hotel_id[0]
# unique_labels

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
                        optimizer,
                        max_lr=1e-3,
                        epochs=args.epoch,
                        steps_per_epoch=len(train_dataloader),
                        div_factor=10,
                        final_div_factor=1,
                        pct_start=0.1,
                        anneal_strategy="cos",
                    )

In [ ]:
IMG_SIZE =224

In [ ]:
acc_top_1, acc_top_5 = [],[]
train_loss, train_score = [],[]
prev_valid_acc = 0
model_name = f"rgb_with_embedding-model-{IMG_SIZE}x{IMG_SIZE}"
counter = 0 
for epoch in trange(1, 20+1):
    training_loss, training_score = trainEpoch(train_dataloader,model, criterion, optimizer, scheduler, epoch,classifier_to_use="rgb")
    train_loss.append(training_loss)
    train_score.append(training_score)
    print(f"train loss : {train_loss} | train_acc : {train_score}")
    val_acc_top_1, val_acc_top_5 = test_classification(validation_dataloader, model,colorFeat="rgb_feats")
    acc_top_1.append(val_acc_top_1)
    acc_top_5.append(val_acc_top_5)
    if prev_valid_acc<val_acc_top_5:
        print("model saved..!!")
        # torch.save(model.state_dict(), "best.pt")
        save_checkpoint(model, scheduler, optimizer, epoch, model_name, train_loss, train_score)
        prev_valid_acc = val_acc_top_5
        counter = 0
    else:
        counter +=1
    if(counter==5):
        print("early stopping applied, training done")
        break

In [ ]:
rgb_metrics_df = pd.DataFrame({"acc_top_1":acc_top_1,"acc_top_5":acc_top_5,"train_loss":train_loss,"train_score":train_score})
rgb_metrics_df.to_csv("/Users/yhemmy/Documents/code/hotel-id-experiments/artefacts/rgb_metrics_df.csv",index=False)

In [ ]:
# test_classification(validation_dataloader, model,colorFeat="rgb_feats")

In [ ]:
acc_top_1, acc_top_5 = [],[]
train_loss, train_score = [],[]
prev_valid_acc = 0
model_name = f"embedding-model-{IMG_SIZE}x{IMG_SIZE}"
counter = 0 
for epoch in trange(1, 20+1):
    training_loss, training_score = trainEpoch(train_dataloader,model, criterion, optimizer, scheduler, epoch,classifier_to_use="embedding")
    train_loss.append(training_loss)
    train_score.append(training_score)
    print(f"train loss : {train_loss} | train_acc : {train_score}")
    val_acc_top_1, val_acc_top_5 = test_classification(validation_dataloader, model)
    acc_top_1.append(val_acc_top_1)
    acc_top_5.append(val_acc_top_5)
    if prev_valid_acc<val_acc_top_5:
        print("model saved..!!")
        # torch.save(model.state_dict(), "best.pt")
        save_checkpoint(model, scheduler, optimizer, epoch, model_name, train_loss, train_score)
        prev_valid_acc = val_acc_top_5
        counter = 0
    else:
        counter +=1
    if(counter==5):
        print("early stopping applied, training done")
        break

In [ ]:
embedding_metrics_df = pd.DataFrame({"acc_top_1":acc_top_1,"acc_top_5":acc_top_5,"train_loss":train_loss,"train_score":train_score})
embedding_metrics_df.to_csv("/Users/yhemmy/Documents/code/hotel-id-experiments/artefacts/embedding_metrics_df.csv",index=False)

In [ ]:
# for epoch in trange(1, 5+1):
#     train_loss, train_score = trainEpoch(train_dataloader,model, criterion, optimizer, scheduler, epoch,classifier_to_use="embedding")
#     print(f"train loss : {train_loss} | train_acc : {train_score}")

In [ ]:
acc_top_1, acc_top_5 = [],[]
train_loss, train_score = [],[]
prev_valid_acc = 0
model_name = f"hist_with_embedding-model-{IMG_SIZE}x{IMG_SIZE}"
counter = 0 
for epoch in trange(1, 20+1):
    training_loss, training_score = trainEpoch(train_dataloader,model, criterion, optimizer, scheduler, epoch,classifier_to_use="hist")
    train_loss.append(training_loss)
    train_score.append(training_score)
    print(f"train loss : {train_loss} | train_acc : {train_score}")
    val_acc_top_1, val_acc_top_5 = test_classification(validation_dataloader, model,colorFeat="hist_feats")
    acc_top_1.append(val_acc_top_1)
    acc_top_5.append(val_acc_top_5)
    if prev_valid_acc<val_acc_top_5:
        print("model saved..!!")
        # torch.save(model.state_dict(), "best.pt")
        save_checkpoint(model, scheduler, optimizer, epoch, model_name, train_loss, train_score)
        prev_valid_acc = val_acc_top_5
        counter = 0
    else:
        counter +=1
    if(counter==5):
        print("early stopping applied, training done")
        break

In [ ]:
hist_metrics_df = pd.DataFrame({"acc_top_1":acc_top_1,"acc_top_5":acc_top_5,"train_loss":train_loss,"train_score":train_score})
hist_metrics_df.to_csv("/Users/yhemmy/Documents/code/hotel-id-experiments/artefacts/hist_metrics_df.csv",index=False)

In [ ]:
acc_top_1, acc_top_5 = [],[]
train_loss, train_score = [],[]
prev_valid_acc = 0
model_name = f"hsv_with_embedding-model-{IMG_SIZE}x{IMG_SIZE}"
counter = 0 
for epoch in trange(1, 20+1):
    training_loss, training_score = trainEpoch(train_dataloader,model, criterion, optimizer, scheduler, epoch,classifier_to_use="hsv")
    train_loss.append(training_loss)
    train_score.append(training_score)
    print(f"train loss : {train_loss} | train_acc : {train_score}")
    val_acc_top_1, val_acc_top_5 = test_classification(validation_dataloader, model,colorFeat="hsv_feats")
    acc_top_1.append(val_acc_top_1)
    acc_top_5.append(val_acc_top_5)
    if prev_valid_acc<val_acc_top_5:
        print("model saved..!!")
        # torch.save(model.state_dict(), "best.pt")
        save_checkpoint(model, scheduler, optimizer, epoch, model_name, train_loss, train_score)
        prev_valid_acc = val_acc_top_5
        counter = 0
    else:
        counter +=1
    if(counter==5):
        print("early stopping applied, training done")
        break

In [ ]:
hsv_metrics_df = pd.DataFrame({"acc_top_1":acc_top_1,"acc_top_5":acc_top_5,"train_loss":train_loss,"train_score":train_score})
hsv_metrics_df.to_csv("/Users/yhemmy/Documents/code/hotel-id-experiments/artefacts/hsv_metrics_df.csv",index=False)

In [ ]:
# for epoch in trange(1, 5+1):
#     train_loss, train_score = trainEpoch(train_dataloader,model, criterion, optimizer, scheduler, epoch,classifier_to_use="hist")
#     print(f"train loss : {train_loss} | train_acc : {train_score}")

In [ ]:
# optimizer.state_dict()

In [ ]:
# from tqdm.notebook import trange, tqdm
# for i in trange(10):
#     print("gbasgbos")

In [ ]:
for epoch in range(1, 5+1):
    train_loss, train_score = trainEpoch(train_dataloader,model, criterion, optimizer, scheduler, epoch,classifier_to_use="hsv")

In [ ]:
# fuse_features =trainWithFuseFeaturesPerEpoch(train_dataloader,model)
h_classes =trainWithFuseFeaturesPerEpoch(train_dataloader,model)

In [ ]:
# print(fuse_features.shape)
print(h_classes)

In [ ]:
# for i in img_id:
#     f = df[df.image_id==i]["rgb_feats"].values[0]
#     f =torch.tensor(f,dtype=torch.float).to(DEVICE)
#     fuse = torch.cat((fuse_features[-1],f))
#     print(f.shape)
#     print(fuse_features[0].shape)
#     print(fuse[-30:])
#     break

# Exract Embedding

In [ ]:
# timm.list_models()